In [9]:
!pip install pydeck geopandas -q

In [2]:
!pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]' -q

In [1]:
%load_ext google.cloud.bigquery

In [2]:
import pydeck as pdk
import geopandas as gpd
from shapely import wkt
import matplotlib
import matplotlib.cm as cm

In [5]:
%%bigquery upcoming_housing
with upcoming_housing as (

    select 
        address_surface_area,
        address_geometry
    from 
        `quantile.analytics.dim_address` dim_address
    where 
        (address_state = 'housing_formed')
        and 
        (address_is_latest = true)
        and 
        (address_function_types like '%woonfunctie%')

),

cluster_ids as (

  select
    *,
    ST_CLUSTERDBSCAN(address_geometry, 50, 20) OVER () AS cluster_id
  from 
    upcoming_housing

),

clusters as (

  select
    count(*) as n_housing,
    ST_CENTROID_AGG(address_geometry) as centroid,
    sum(address_surface_area) as total_surface_area,
    cluster_id
  from 
    cluster_ids
  where 
    cluster_id is not null
  group by
    cluster_id

)

select 
    *, 
    st_x(centroid) as long, 
    st_y(centroid) as lat
from 
    clusters 

Downloading: 100%|██████████| 1864/1864 [00:01<00:00, 1848.39rows/s]


In [8]:
%%bigquery poi
select
    poi_name,
    poi_type,
    st_x(poi_geometry) as long,
    st_y(poi_geometry) as lat,
    poi_geometry
from 
    `quantile.analytics.fct_poi`
where 
    poi_type in ('supermarket', 'restaurant', 'fast_food', 'cafe', 'hairdresser', 'pub', 'bakery', 'beauty', 'convenience', 'sports', 'electronics')

Downloading: 100%|██████████| 37657/37657 [00:01<00:00, 31374.57rows/s]


In [6]:
upcoming_housing.head()

,n_housing,centroid,total_surface_area,cluster_id,long,lat
0,965,POINT(4.90398191475263 52.3869573181951),79864,0,4.903982,52.386957
1,802,POINT(4.95435829724871 52.3687093368319),65402,1,4.954358,52.368709
2,36,POINT(4.8068002820286 52.2482345300849),2571,2,4.806800,52.248235
3,208,POINT(4.9704021364796 52.3370137535944),15450,3,4.970402,52.337014
4,27,POINT(5.72809549137642 51.8064509567714),3269,4,5.728095,51.806451


In [11]:
poi['icon'] = 'https://cdn1.iconfinder.com/data/icons/grocery-store-filled/64/Grocery_Shop-36-512.png'
poi.head()

,poi_name,poi_type,long,lat,poi_geometry,icon
0,De Kruin,pub,4.501368,51.989036,POINT(4.5013683 51.9890357),https://cdn1.iconfinder.com/data/icons/grocery...
1,Eetcafé The Bottom,pub,5.760809,52.711047,POINT(5.760809 52.7110469),https://cdn1.iconfinder.com/data/icons/grocery...
2,Café Stynsgea,pub,6.159047,53.216426,POINT(6.1590472 53.2164258),https://cdn1.iconfinder.com/data/icons/grocery...
3,Cafe Havana,pub,4.772214,51.821942,POINT(4.772214 51.8219416),https://cdn1.iconfinder.com/data/icons/grocery...
4,Spek & Bonen,pub,4.771803,51.821641,POINT(4.7718026 51.8216408),https://cdn1.iconfinder.com/data/icons/grocery...


In [7]:
upcoming_housing.to_csv('data/upcoming_housing.csv', index=False)

In [12]:
poi.to_csv('data/poi.csv', index=False)